# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])

from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

import re
import numpy as np
import pandas as pd

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report

from sqlalchemy import create_engine
from sklearn.multioutput import MultiOutputClassifier

import pickle

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/susiecrone/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/susiecrone/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/susiecrone/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [3]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

In [3]:
def load_data():
    engine = create_engine('sqlite:///data.db')
    df = pd.read_sql("SELECT * FROM data", engine)
    X = df.message.values
    y = df.loc[:, 'related':'direct_report'].values
    return X, y

### 2. Write a tokenization function to process your text data

In [5]:
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

<>:1: SyntaxWarning: invalid escape sequence '\('
<>:1: SyntaxWarning: invalid escape sequence '\('
/var/folders/sl/bq_xqjss6hb0b96r7d9_7wmr0000gn/T/ipykernel_82163/3411073879.py:1: SyntaxWarning: invalid escape sequence '\('
  url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'


In [7]:
def tokenize(text):
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

In [9]:
# test out function
X, y = load_data()

for message in X[:5]:
    tokens = tokenize(message)
    print(message)
    print(tokens, '\n')


Weather update - a cold front from Cuba that could pass over Haiti
['weather', 'update', '-', 'a', 'cold', 'front', 'from', 'cuba', 'that', 'could', 'pas', 'over', 'haiti'] 

Is the Hurricane over or is it not over
['is', 'the', 'hurricane', 'over', 'or', 'is', 'it', 'not', 'over'] 

Looking for someone but no name
['looking', 'for', 'someone', 'but', 'no', 'name'] 

UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately.
['un', 'report', 'leogane', '80-90', 'destroyed', '.', 'only', 'hospital', 'st.', 'croix', 'functioning', '.', 'needs', 'supply', 'desperately', '.'] 

says: west side of Haiti, rest of the country today and tonight
['say', ':', 'west', 'side', 'of', 'haiti', ',', 'rest', 'of', 'the', 'country', 'today', 'and', 'tonight'] 



### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [11]:
def build_model():
    pipeline = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),

            ('starting_verb', StartingVerbExtractor())
        ])),

        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])

    parameters = {
       # 'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2)),
       # 'clf__estimator__n_estimators': [50, 100, 200],
        'clf__estimator__min_samples_split': [2, 3, 4]       
        
    }
    
    cv = GridSearchCV(pipeline, param_grid=parameters)
    return cv

pipeline.get_params().keys()

dict_keys(['memory', 'steps', 'features', 'clf', 'features__n_jobs', 'features__transformer_list', 'features__transformer_weights', 'features__text_pipeline', 'features__starting_verb', 'features__text_pipeline__memory', 'features__text_pipeline__steps', 'features__text_pipeline__vect', 'features__text_pipeline__tfidf', 'features__text_pipeline__vect__analyzer', 'features__text_pipeline__vect__binary', 'features__text_pipeline__vect__decode_error', 'features__text_pipeline__vect__dtype', 'features__text_pipeline__vect__encoding', 'features__text_pipeline__vect__input', 'features__text_pipeline__vect__lowercase', 'features__text_pipeline__vect__max_df', 'features__text_pipeline__vect__max_features', 'features__text_pipeline__vect__min_df', 'features__text_pipeline__vect__ngram_range', 'features__text_pipeline__vect__preprocessor', 'features__text_pipeline__vect__stop_words', 'features__text_pipeline__vect__strip_accents', 'features__text_pipeline__vect__token_pattern', 'features__text_pipeline__vect__tokenizer', 'features__text_pipeline__vect__vocabulary', 'features__text_pipeline__tfidf__norm', 'features__text_pipeline__tfidf__smooth_idf', 'features__text_pipeline__tfidf__sublinear_tf', 'features__text_pipeline__tfidf__use_idf', 'clf__estimator__bootstrap', 'clf__estimator__class_weight', 'clf__estimator__criterion', 'clf__estimator__max_depth', 'clf__estimator__max_features', 'clf__estimator__max_leaf_nodes', 'clf__estimator__min_impurity_decrease', 'clf__estimator__min_impurity_split', 'clf__estimator__min_samples_leaf', 'clf__estimator__min_samples_split', 'clf__estimator__min_weight_fraction_leaf', 'clf__estimator__n_estimators', 'clf__estimator__n_jobs', 'clf__estimator__oob_score', 'clf__estimator__random_state', 'clf__estimator__verbose', 'clf__estimator__warm_start', 'clf__estimator', 'clf__n_jobs'])

In [12]:
def display_results(cv, y_test, y_pred):
    labels = np.unique(y_pred)
    accuracy = (y_pred == y_test).mean()

    print("Labels:", labels)
    print("Accuracy:", accuracy)
    print("\nBest Parameters:", cv.best_params_)

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

def main():
    X, y = load_data()
    X_train, X_test, y_train, y_test = train_test_split(X, y)

    model = build_model()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    display_results(model, y_test, y_pred)
    
    
main()

In [23]:
X, y = load_data()
X_train, X_test, y_train, y_test = train_test_split(X, y)

model = build_model()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

display_results(model, y_test, y_pred)

/opt/anaconda3/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packa

Labels: [0. 1. 2.]
Accuracy: 0.9274988129154795

Best Parameters: {'clf__estimator__min_samples_split': 2}


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [27]:
# Converting y_test and y_pred to a binary indicator format
y_test_bin = (y_test > 0).astype(int)
y_pred_bin = (y_pred > 0).astype(int)

# Generate the classification report
report = classification_report(y_test_bin, y_pred_bin, target_names=[f'Class {i}' for i in range(y_test.shape[1])])
print(report)

              precision    recall  f1-score   support

     Class 0       0.77      0.99      0.87      5005
     Class 1       0.42      0.02      0.04      1089
     Class 2       0.00      0.00      0.00        31
     Class 3       0.49      0.14      0.22      2688
     Class 4       0.00      0.00      0.00       505
     Class 5       0.07      0.00      0.01       335
     Class 6       0.00      0.00      0.00       177
     Class 7       0.00      0.00      0.00       120
     Class 8       0.00      0.00      0.00       211
     Class 9       0.00      0.00      0.00         0
    Class 10       0.13      0.00      0.01       408
    Class 11       0.22      0.01      0.01       736
    Class 12       0.18      0.00      0.01       582
    Class 13       0.00      0.00      0.00        94
    Class 14       0.00      0.00      0.00       154
    Class 15       0.00      0.00      0.00        66
    Class 16       0.00      0.00      0.00       195
    Class 17       0.00    

/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-pac

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = 

cv = 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

# Ensure you have StartingVerbExtractor and tokenize function implemented
def build_model():
    # Create a pipeline with feature extraction and classification
    pipeline = Pipeline([
        ('features', FeatureUnion([
            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),  # Convert text to vector
                ('tfidf', TfidfTransformer())                    # Apply TF-IDF
            ])),
            ('starting_verb', StartingVerbExtractor())          # Custom feature extraction
        ])),
        ('clf', MultiOutputClassifier(LogisticRegression()))   # Replace RandomForest with Logistic Regression
    ])

    # Set parameters for Grid Search
    parameters = {
        # Uncomment these lines to tune more hyperparameters
        # 'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2)),  # Unigrams or bigrams
        # 'clf__estimator__C': [0.1, 1.0, 10.0],                           # Inverse of regularization strength
        'clf__estimator__solver': ['lbfgs', 'liblinear'],  # Optimizers to try
    }
    
    # Create a GridSearchCV object to find the best parameters
    cv = GridSearchCV(pipeline, param_grid=parameters, verbose=3, n_jobs=-1)
    
    return cv

In [ ]:
def display_results(cv, y_test, y_pred):
    labels = np.unique(y_pred)
    accuracy = (y_pred == y_test).mean()

    print("Labels:", labels)
    print("Accuracy:", accuracy)
    print("\nBest Parameters:", cv.best_params_)

In [ ]:
X, y = load_data()
X_train, X_test, y_train, y_test = train_test_split(X, y)

model = build_model()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

display_results(model, y_test, y_pred)

In [ ]:
# Converting y_test and y_pred to a binary indicator format
y_test_bin = (y_test > 0).astype(int)
y_pred_bin = (y_pred > 0).astype(int)

# Generate the classification report
report = classification_report(y_test_bin, y_pred_bin, target_names=[f'Class {i}' for i in range(y_test.shape[1])])
print(report)

### 9. Export your model as a pickle file

In [29]:
# Save model to pickle file
with open('trained_model.pkl', 'wb') as file:
    pickle.dump(model, file)

#Code to load it back
#Load the model from the pickle file
with open('trained_model.pkl', 'rb') as file:
    loaded_model = pickle.load(file)

#Use for predictions
y_pred = loaded_model.predict(X_test)

### 10. Use this notebook to complete `train_classifier.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

In [33]:
!pip install numpy==1.16.5
!pip install scikit-learn==0.19.1 --no-build-isolation

  Using cached numpy-1.16.5.zip (5.1 MB)
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [17 lines of output]
      Running from numpy source directory.
      /private/var/folders/sl/bq_xqjss6hb0b96r7d9_7wmr0000gn/T/pip-install-f0aihhyw/numpy_281be8fa47f54c8cad03fb379fa6ae78/numpy/distutils/misc_util.py:476: SyntaxWarning: "is" with 'str' literal. Did you mean "=="?
        return is_string(s) and ('*' in s or '?' is s)
      Traceback (most recent call last):
        File "<string>", line 2, in <module>
        File "<pip-setuptools-caller>", line 34, in <module>
        File "/private/var/folders/sl/bq_xqjss6hb0b96r7d9_7wmr0000gn/T/pip-install-f0aihhyw/numpy_281be8fa47f54c8cad03fb379fa6ae78/setup.py", line 419, in <module>
          setup_package()
        File "/private/var/folders/sl/bq_xqjss6hb0b96r7d9_7wmr0000gn/T/pip-install-f0aihhyw/numpy_281be8fa47f54c8cad03fb379f

In [35]:
!pip install --upgrade pip setuptools wheel

  Using cached pip-25.0.1-py3-none-any.whl.metadata (3.7 kB)
  Using cached wheel-0.45.1-py3-none-any.whl.metadata (2.3 kB)
Using cached pip-25.0.1-py3-none-any.whl (1.8 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 19.2 MB/s eta 0:00:00
Using cached wheel-0.45.1-py3-none-any.whl (72 kB)
  Attempting uninstall: wheel
    Found existing installation: wheel 0.44.0
    Uninstalling wheel-0.44.0:
      Successfully uninstalled wheel-0.44.0
  Attempting uninstall: setuptools
    Found existing installation: setuptools 75.1.0
    Uninstalling setuptools-75.1.0:
      Successfully uninstalled setuptools-75.1.0
  Attempting uninstall: pip
    Found existing installation: pip 24.2
    Uninstalling pip-24.2:
      Successfully uninstalled pip-24.2
